In [41]:
import matplotlib.pyplot as plt
from PriorityQueue import PriorityQueue
from Cell import Position, Cell

from matplotlib.animation import FuncAnimation
from IPython.display import HTML

from typing import Iterable, Optional

def chebyshev_move(start: Position, end: Position) -> tuple[list[int], list[int]]:
    cur_x, cur_y = start
    xs = [cur_x]
    ys = [cur_y]

    while (cur_x != end.x) or (cur_y != end.y):
        dx = end.x - cur_x
        dy = end.y-cur_y
        if abs(dx) == abs(dy):
            # cur_x > end.x -> decrease x
            if dx < 0: cur_x -= 1
            else: cur_x += 1

            #cur_y > end.y -> decrease y
            if dy < 0: cur_y -= 1
            else: cur_y += 1
        elif abs(dx) < abs(dy):
            if dy < 0: cur_y -= 1
            elif dy > 0: cur_y += 1
        else:
            if dx < 0: cur_x -= 1
            elif dx > 0: cur_x += 1
        xs.append(cur_x)
        ys.append(cur_y)
    return xs, ys

def path_traceback(start_state: Cell, goal_state: Cell) -> list[Cell]:
    path = []
    while goal_state != start_state:
        path.append(goal_state)
        goal_state = goal_state.parent
    path.append(start_state)
    return list(reversed(path))

def astar_vacuum(grid_dim: tuple[int, int],
                 dirty_cells: Iterable[Position],
                 start: Position, *,
                 do_traceback: bool = False)\
                -> tuple[Cell, Optional[list[Cell]]]:
    my_queue: PriorityQueue[Cell] = PriorityQueue()

    start_node = Cell(position=start, grid_dim=grid_dim, dirty_cells=dirty_cells)
    start_node.calc_cost()
    my_queue.push(start_node)
    while my_queue:
        cur = my_queue.pop()

        if len(cur.dirty_cells) == 0: break
        
        for neighbour in cur.expand_cell():
            if my_queue.get_attr(neighbour, 'cost', default_value=neighbour.cost + 1) > neighbour.cost:
                my_queue.push(neighbour)

    traceback = path_traceback(start_node, cur) if do_traceback else None
    return cur, traceback

def show_animation(grid_dim:tuple[int, int], dirty_cells: set[Position],
                   xs: list[int], ys: list[int],
                   figure = plt.gcf()) -> None:
    ax = figure.add_subplot(xlim=(1, grid_dim[1] + 1), ylim=(1, grid_dim[0] + 1))
    ax.set_aspect('equal')
    ax.grid(which='major')
    for tick in ax.xaxis.get_major_ticks():
        tick.tick1line.set_visible(False)
        tick.tick2line.set_visible(False)
        tick.label1.set_visible(False)
        tick.label2.set_visible(False)
    for tick in ax.yaxis.get_major_ticks():
        tick.tick1line.set_visible(False)
        tick.tick2line.set_visible(False)
        tick.label1.set_visible(False)
        tick.label2.set_visible(False)

    robot, = ax.plot([], [], 's')
    dirty, = ax.plot([cell.x + 0.5 for cell in dirty_cells],
                     [cell.y + 0.5 for cell in dirty_cells], 'Xr')

    clean_cost = -3
    cur_cost = -4
    cost_format = lambda clean, cur: f'Clean cost: {clean}\nCurrent cost: {cur}'
    cost_txt = ax.text(6, 8.2, cost_format(clean_cost, cur_cost))
    first = True
    def animate(i):
        nonlocal dirty_cells, clean_cost, cur_cost, first
        
        if (xs[i], ys[i]) in dirty_cells and not first:
            cur_cost += clean_cost
            dirty_cells = dirty_cells - {(xs[i], ys[i])}
            dirty.set_data([cell.x + 0.5 for cell in dirty_cells],
                           [cell.y + 0.5 for cell in dirty_cells])
            first = True
        else:
            if (xs[i], ys[i]) in dirty_cells: first = False
            clean_cost += 1
            cur_cost += 1
        cost_txt.set_text(cost_format(clean_cost, cur_cost))
        robot.set_data([xs[i] + 0.5], [ys[i] + 0.5])
        return robot, cost_txt, dirty

    ani = FuncAnimation(figure, animate, len(ys), interval=500, blit=True)

    return HTML(ani.to_jshtml())

<Figure size 640x480 with 0 Axes>

In [8]:
dirty = {Position(1, 1),
         Position(1, 6),
         Position(3, 8),
         Position(5, 8),
         Position(8, 6)}
start = Position(4, 5)
grid_dim = (8, 8)

In [3]:
goal = astar_vacuum(grid_dim, dirty, start, do_traceback=True)
print(f"Total cost: {goal[0].cost}")

Total cost: 64


In [4]:
for i, cell in enumerate(goal[1]):
    action = 'Start at' if i == 0 else 'Suck'
    print(f"{action} cell {cell.position} with cost {cell.cost}")

Start at cell (4, 5) with cost 0
Suck cell (1, 6) with cost 7
Suck cell (3, 8) with cost 15
Suck cell (5, 8) with cost 25
Suck cell (8, 6) with cost 39
Suck cell (1, 1) with cost 64


In [32]:
xs = []
ys = []
for i, cell in enumerate(goal[1][:-1]):
    temp_x, temp_y = chebyshev_move(cell.position, goal[1][i + 1].position)
    xs.extend(temp_x)
    ys.extend(temp_y)
xs.append(goal[0].position.x)
ys.append(goal[0].position.y)

In [42]:
show_animation(grid_dim, dirty, xs, ys)